# This notebook takes a pdb file and identifies intersubunit interactions within a user-chosen radius in angstroms. The file output is a .pml text file that can be run in the PyMol command line with the '@' command.

In [20]:
import numpy as np
import string

In [2]:
pdbNAME = input('Type filename here (include .pdb): ')

Type filename here (include .pdb): 8dwk.pdb


In [3]:
angsLimit = input('Type distance limit in angstroms: ')

Type distance limit in angstroms: 3


In [25]:
chainInput = list(input('Type the chains you want to include in the analysis (separated by commas): ').upper())
chainList = [n for n in chainInput if n in list(string.ascii_uppercase)]

Type the chains you want to include in the analysis (separated by commas): a, b, c


In [27]:
# /6njm//C/GLN`28/CA
pmbCode = pdbNAME[0:4]
fileName = pmbCode + '_intersubDistance.pml'
pmlFile = open(fileName, 'w')
pmlFile = open(fileName, 'a')

In [28]:
openedPDB = open(pdbNAME)
readPDB = openedPDB.read()

PDBlines = readPDB.split('\n')

atomXYZ_chain = {}
for line in PDBlines:

    if line[0:4] =='ATOM':
        if line[21] in atomXYZ_chain: #if line[21] is a key in dic atomXYZ_chain:
            atomXYZ_chain[line[21]].append(line) #append that dic value, which should be a list, with the line
        else:         #else make a new key with value = CHAIN with the list containing the line
            atomXYZ_chain[line[21]] = [line]
#now you have a dictionary with keys of chains and values for each key(chain) of lines of that chain
#ex. print(atomXYZ_chain['A']) gives you a list of the PDB lines for that chain

print(type(list(atomXYZ_chain.keys())))
for chain0 in chainList: #for every chain ['A', 'B', 'C', 'D']
    for lineN in atomXYZ_chain[chain0]: #for every item in list, i.e. every PDB line (atom) in the chain's list
        #print(lineN)
        x0 = float(lineN[31:38])
        y0 = float(lineN[39:46])
        z0 = float(lineN[47:55])
        
        for chain1 in ['A', 'B', 'C', 'D']:
            if chain1 != chain0:
                for lineM in atomXYZ_chain[chain1]:
                    x1 = float(lineM[31:38])
                    y1 = float(lineM[39:46])
                    z1 = float(lineM[47:55])
                    xzDist = (((z1-z0)**2)+((y1-y0)**2))**.5
                    xyzDist = np.sqrt((xzDist)**2+(x1-x0)**2)
                    if xyzDist < float(angsLimit):
                        print(xyzDist, lineN[0:22],'----- ', lineM[0:22])
                        
                        
                        #below writes to the pml file that you want to run in pymol command line to generate bond distance labels
                        pmlFile.write('dist' + ' ' + 'dist' + '_' + lineN[23:31].replace(' ', '') + '_' + lineM[23:31].replace(' ','') + ','  + '/' + pmbCode + '//' + lineN[21] + '/' + lineN[17:20] + '`' + lineN[23:31].replace(' ', '') + '/' + lineN[13:15].replace(' ', '')+ ',' + '/' + pmbCode + '//' + lineM[21] + '/' + lineM[17:20] + '`' + lineM[23:31].replace(' ', '') + '/' + lineM[13:15].replace(' ', '')+ '\n')
                        
                        
                        
                        
                        
            elif chain1 == chain0:
                break
print('END')

<class 'list'>
2.9324216954592295 ATOM   3387  NH1 ARG B -----  ATOM   1755  O   ARG A
2.612060680765284 ATOM   3435  OH  TYR B -----  ATOM   1747  O   ASP A
2.7293940719507725 ATOM   3467  NZ  LYS B -----  ATOM   1747  O   ASP A
2.787276269048333 ATOM   4089  O   ASP B -----  ATOM   1089  OH  TYR A
2.9628138652301472 ATOM   4089  O   ASP B -----  ATOM   1121  NZ  LYS A
2.9827058185479833 ATOM   4097  O   ARG B -----  ATOM   1041  NH1 ARG A
2.847861127232154 ATOM   4716  O   SER C -----  ATOM    918  ND2 ASN A
2.8841567225100646 ATOM   4727  N   THR C -----  ATOM    905  O   GLU A
2.482167601109966 ATOM   4730  O   THR C -----  ATOM    393  N   GLU A
2.571272253185182 ATOM   4742  N   LYS C -----  ATOM    379  O   GLU A
2.6087075343932282 ATOM   4754  O   LYS C -----  ATOM   1349  NZ  LYS A
2.8498987701320204 ATOM   4759  NZ  LYS C -----  ATOM   2264  O   GLU A
2.9159180372568794 ATOM   4779  O   GLU C -----  ATOM   2289  N   CYS A
2.8763801904477098 ATOM   4830  O   VAL C -----  ATOM 

In [29]:
pmlFile.close()